In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import matplotlib.ticker as ticker

from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from IPython.display import display, Markdown

In [ ]:
df = pd.read_csv("../lego.population.csv", sep = ",", encoding = "latin1")

In [ ]:
# Rensking av datasett
df2 = df[['Set_Name', 'Theme', 'Pieces', 'Price', 'Pages', 'Minifigures', 'Unique_Pieces', 'Ages']]
df2 = df2.dropna()
df2['Theme'] = df2['Theme'].astype(str).str.replace(r'[^a-zA-Z0-9\s-]', '', regex = True)
df2['Price'] = df2['Price'].str.replace('\$', '', regex = True).astype(float)
df = False

# 1. Bli kjent med dataene

## Scatterplot av alle mulige forklaringsvariabler

In [ ]:
# Scatterplot av alle variabler
colors=['red', 'orange', 'green', 'darkblue']

for i, field in enumerate(["Pages", "Pieces", "Unique_Pieces", "Minifigures"]):
    display(Markdown(f"### {field}"))
    plt.scatter(df2[field], df2['Price'], label='Data Points', color=colors[i])
    plt.xlabel(field)
    plt.ylabel('Price')
    plt.title('Kryssplott med regresjonslinje (enkel LR)')
    plt.legend()
    plt.grid()
    plt.show()


## Dyreste brikker, dyreste sider

In [ ]:
fields = ["Pieces", "Pages", "Unique_Pieces"]

for field in fields:
    df2[f"Price_Per_{field}"] = df2['Price'] / df2[field]

for field in fields:
    display(Markdown(f'## Sorted by Price per {field}'))
    display(df2.sort_values(f'Price_Per_{field}'))
    
    plt.hist(df2[f'Price_Per_{field}'], bins=50, color='skyblue', edgecolor='black')
    plt.xlabel(f'Price/{field} [$]')
    
    plt.yscale('log')
    y_ax = plt.gca().yaxis
    y_ax.set_major_locator(ticker.FixedLocator([1, 10, 100, 1000]))
    y_ax.set_major_formatter(ticker.ScalarFormatter())
    
    plt.ylabel('')
    #plt.gca().set_aspect(1)
    plt.show()


# 2. Spesifisere en matematisk modell

Vi ser en trend i `Pieces`, `Pages`, og `Unique_Pieces` ift. pris. Variasjonene i disse ser noe normalfordelt ut i log-skala. Vi prøver enkel lineær regresjon med disse.

In [ ]:

for field in ["Pages", "Pieces", "Unique_Pieces"]:
    model4a = smf.ols(f"Price ~ {field}", data = df2)
    fit = model4a.fit()

    display(Markdown(f"### {field}"))
    display(fit.summary())

    regression_x = np.array(df2[field])
    regression_y = fit.params[field] * regression_x + fit.params[field]
    
    plt.scatter(df2[field], df2['Price'], label='Data Points')
    plt.plot(regression_x, regression_y, color='red', label='Regression Line')
    plt.xlabel(field)
    plt.ylabel('Price')
    plt.title('Kryssplott med regresjonslinje (enkel LR)')
    plt.legend()
    plt.grid()
    plt.show()

    figure, axis = plt.subplots(1, 2, figsize = (15, 5))
    sns.scatterplot(x = fit.fittedvalues, y = fit.resid, ax = axis[0])
    axis[0].set_ylabel("Residual")
    axis[0].set_xlabel("Predikert verdi")
    
    sm.qqplot(fit.resid, line = '45', fit = True, ax = axis[1])
    axis[1].set_ylabel("Kvantiler i residualene")
    axis[1].set_xlabel("Kvantiler i normalfordelingen")
    plt.show()

Vi ser at bredden på området for residualer ikke er konstant, så vi prøver å transformere datasettet med `math.log`

In [ ]:
# Gir feilmelding dersom vi transformerer mer en én gang
assert "did_transform" not in locals()

for field in ["Price", "Pages", "Pieces", "Unique_Pieces", "Minifigures"]:
    df2[field] = df2[field].apply(math.log)

did_transform = True

Vi lager nye modeller med de transformerte dataene

In [ ]:

for field in ["Pages", "Pieces", "Unique_Pieces"]:
    model4a = smf.ols(f"Price ~ {field}", data = df2)
    fit = model4a.fit()

    display(Markdown(f"### {field}"))
    display(fit.summary())

    regression_x = np.array(df2[field])
    regression_y = fit.params[field] * regression_x + fit.params[field]
    
    plt.scatter(df2[field], df2['Price'], label='Data Points')
    plt.plot(regression_x, regression_y, color='red', label='Regression Line')
    plt.xlabel(field)
    plt.ylabel('Price')
    plt.title('Kryssplott med regresjonslinje (enkel LR)')
    plt.legend()
    plt.grid()
    plt.show()

    figure, axis = plt.subplots(1, 2, figsize = (15, 5))
    sns.scatterplot(x = fit.fittedvalues, y = fit.resid, ax = axis[0])
    axis[0].set_ylabel("Residual")
    axis[0].set_xlabel("Predikert verdi")
    
    sm.qqplot(fit.resid, line = '45', fit = True, ax = axis[1])
    axis[1].set_ylabel("Kvantiler i residualene")
    axis[1].set_xlabel("Kvantiler i normalfordelingen")
    plt.show()

Vi ser at QQ-plottene er mye bedre, og `Pieces` har den beste `Adj. R-squared`.
`Price ~ Pieces` er den beste modellen når man ikke tar hensyn til lisens.

Vi skal nå se om lisens har en påvirkning på pris.

In [ ]:
# Lager dummy-variabler for hver av kategoriene under

licensed = {
    'Spider-Man', 'Powerpuff Girls', 'Minions', 'Stranger Things', 'Star Wars', 'Marvel', 'Disney', 'Harry Potter', 'Minecraft',
    'Jurassic World', 'Batman', 'DC', 'Trolls World Tour', 'Overwatch', 'LEGO Frozen 2'
}

not_licensed = {
    'Monkie Kid', 'Friends', 'City', 'NINJAGO', 'DUPLO', 'Creator 3-in-1', 'Hidden Side', 'Ideas', 'Classic', 'Powered UP'
}

uncertain = {
    'Unikitty', 'Minifigures', 'THE LEGO MOVIE 2', 'Speed Champions', 'Juniors', 'Creator Expert'
}

# verifiser at vi har fått med alle temaene i gruppene
assert licensed|uncertain|not_licensed == set(df2['Theme'])

df2['Category'] = np.where(
    df2['Theme'].isin(
        licensed
    ),
    'licensed',
    np.where(
        df2['Theme'].isin(
            not_licensed
        ),
        'not_licensed',
        np.where(
            df2['Theme'].isin(
                uncertain   
            ),
            'uncertain',
            'unaccounted for'
        )
    )
)


## Generering og presentasjon av modell

In [ ]:
model = smf.ols(f'Price ~ Pieces * Category' , data = df2)
fit   = model.fit()
display(fit.summary())

# Display linear regression
intercept = [fit.params['Category[T.not_licensed]'],        fit.params['Category[T.uncertain]'],        0] + fit.params['Intercept']
slope     = [fit.params['Pieces:Category[T.not_licensed]'], fit.params['Pieces:Category[T.uncertain]'], 0] + fit.params['Pieces']

for i, theme in enumerate([not_licensed, uncertain, licensed]):
    subset = df2[df2['Theme'].isin(theme)]
    regression_x = np.array(subset['Pieces'])
    regression_y = slope[i] * regression_x + intercept[i]

    # Plot scatter plot and regression line
    plt.scatter(subset['Pieces'], subset['Price'], color=plt.cm.tab10(i))
    plt.plot(regression_x, regression_y, color=plt.cm.tab10(i), label=theme)

# Display
display(Markdown('### Kryssplot'))
plt.show()

# QQ og residualplot
figure, axis = plt.subplots(1, 2, figsize = (15, 5))
sns.scatterplot(x = fit.fittedvalues, y = fit.resid, ax = axis[0])
axis[0].set_ylabel("Residual")
axis[0].set_xlabel("Predikert verdi")
sm.qqplot(fit.resid, line = '45', fit = True, ax = axis[1])
axis[1].set_ylabel("Kvantiler i residualene")
axis[1].set_xlabel("Kvantiler i normalfordelingen")

display(Markdown('### QQ og residualplot'))
plt.show()

```
                                      coef  std err       t  P>|t|  [0.025  0.975]
Pieces:Category[T.not_licensed]    -0.0451    0.034  -1.342  0.180  -0.111  0.021
Pieces:Category[T.uncertain]        0.0868    0.051   1.698  0.090  -0.014  0.187
```
Vi ser at p-verdiene til `not_licensed` og `uncertain` ikke er gode nok til å forkaste nullhypotesen, vi kan derfor ikke si noe om hvorvidt lisens påvirker pris.

# 3. Initialisere og tilpasse modellen

# 4. Presentere resultater fra den tilpassede modellen

# 5. Evaluere om modellen passer til dataene